In [1]:
cd ..

/Users/joeranbosma/stack/Projects/M5Forecast/uncertainty


In [2]:
cd ..

/Users/joeranbosma/stack/Projects/M5Forecast


In [3]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os, gc
import matplotlib.pyplot as plt
from tqdm import tqdm

from tensorflow.keras.layers import (Dense, Dropout, Flatten, Input, BatchNormalization, Lambda, 
                                     concatenate, Embedding, Reshape)
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras import backend as K
from tensorflow.keras.utils import Sequence
from tensorflow.keras.callbacks import Callback, LearningRateScheduler

import wandb
from wandb.keras import WandbCallback

# own imports
from evaluation import Referee, load_data, select_dates, select_day_nums
from train import BatchCreator, Logger, plot_confidence_series, plot_some_confidence_intervals
from model_builder import (get_pinball_losses, get_simple_dist_model, get_simple_dense_model,
                           get_variable_dist_model, get_direct_dist_model)
from flow import model_predict, denorm_preds, warp_preds_to_ref_form, plot_some, evaluate_model
from preprocess import preprocess, get_features, pandas_cat_data, reset_categorical_features


# Setup file paths
data_dir = 'data/'
os.environ['DATA_DIR'] = data_dir
sub_dir = 'submissions_uncertainty/'
plt.rcParams['font.size'] = 13

# Hardcode requested quantiles
quantiles = [0.005, 0.025, 0.165, 0.25, 0.5, 0.75, 0.835, 0.975, 0.995]

# Combine predictions of all levels

In [4]:
# Load in the data set
calendar, sales_train_validation, sell_prices = load_data()

sales_true = select_dates(sales_train_validation, day_end=1913, num_days=28, include_metadata=True)
sales_train = select_dates(sales_train_validation, day_start=1, num_days=1913-28, include_metadata=True)

ref = Referee(sales_true, sales_train, sell_prices, calendar, verbose=True)

Initializing Referee
Converting true sales to quantile form
Calculating weights for each level...
Calculating scale for each level...
Finished setup.


## Setup

For each level:  

1. Read the (prepared) dataset
2. Get the validation batch creator
3. Get the denormalised predictions
4. Save to file

Then:

1. Combine predictions

In [7]:
# Set input sizes for the model
input_shapes = {
    1: (178,),
    2: (181,),
    3: (191,),
    4: (181,),
    5: (188,),
    6: (184,),
    7: (191,),
    8: (194,),
    9: (201,),
    10: (3237,),
    11: (3240,),
    12: (3255,),
}

# select whether to perform the category-reset step of the preprocessing
# reset_categories = {d: False if d in [3] else True for d in range(1, 1+12)}
reset_categories = {d: True for d in range(1, 1+12)}

# select which model function was used to build the model
models_base = {
    1: get_simple_dist_model(inp_shape=input_shapes[1], num_nodes=2, clear_session=False),# Dist(2)!
    2: get_direct_dist_model(inp_shape=input_shapes[2], num_nodes=3, clear_session=False),
    3: get_direct_dist_model(inp_shape=input_shapes[3], num_nodes=6, clear_session=False),
    4: get_direct_dist_model(inp_shape=input_shapes[4], num_nodes=6, clear_session=False),
    5: get_direct_dist_model(inp_shape=input_shapes[5], num_nodes=8, clear_session=False),
    6: get_direct_dist_model(inp_shape=input_shapes[6], num_nodes=8, clear_session=False),
    7: get_direct_dist_model(inp_shape=input_shapes[7], num_nodes=6, clear_session=False),
    8: get_direct_dist_model(inp_shape=input_shapes[8], num_nodes=10, clear_session=False),
    9: get_direct_dist_model(inp_shape=input_shapes[9], num_nodes=16, clear_session=False),
   10: get_direct_dist_model(inp_shape=input_shapes[10], num_nodes=256, clear_session=False),
   11: get_direct_dist_model(inp_shape=input_shapes[11], num_nodes=256, clear_session=False),
   12: get_direct_dist_model(inp_shape=input_shapes[12], num_nodes=512, clear_session=False),
}

# select the trained models
model_path = 'models/uncertainty/'
models_weights = {
    1: model_path + 'train_multiple_levels/run1/level1_stepped_lr_part3_WSPL0.055553566486644584.h5',
    2: model_path + 'train_multiple_levels/run2/level2_stepped_lr_part3_WSPL9.30e-02.h5',
    3: model_path + 'training_scheme/try1/level3_nodes6_part1_WSPL1.05e-01.h5',
    4: model_path + 'train_multiple_levels/run3/level4_stepped_lr_part3_WSPL7.16e-02.h5',
    5: model_path + 'train_multiple_levels/run3/level5_stepped_lr_part3_WSPL9.83e-02.h5',
    6: model_path + 'train_multiple_levels/run3/level6_stepped_lr_part3_WSPL9.16e-02.h5',
    7: model_path + 'train_multiple_levels/run2/level7_stepped_lr_part3_WSPL1.36e-01.h5',
    8: model_path + 'train_multiple_levels/run3/level8_stepped_lr_part2_WSPL1.36e-01.h5',
    9: model_path + 'train_multiple_levels/run4/level9_stepped_lr_part3_WSPL1.47e-01.h5',
   10: model_path + 'hyper_opt/level_10/model_direct_dist_256_manual_lr_to_1e-3_finetune_bs5k.h5',
   11: model_path + 'hyper_opt/level_11/direct256_stepped_lr_part2.h5',
   12: model_path + 'training_scheme/try1/best_model_level12.h5',
}

In [8]:
collected_dfs = {}

for level in range(1, 12+1):
    print("Starting level {}".format(level))
    ###################
    #### READ DATA ####
    ###################
    prediction_lag = 28
    
    
    fn = data_dir + 'eval_prep/level_{}_simple_fe_2016_04_24_2016_05_22_normalised_demand_lag_{}.pickle'.format(
        level, prediction_lag)

    # check if already preprocessed
    assert os.path.exists(fn), "Run preprocess_validation_features.ipynb first"
    
    print("Reading from file..")
    data = pd.read_pickle(fn)

    # select features
    features = get_features(level=level, prediction_lag=prediction_lag, 
                            sell_price_features=('sell_price' in data.columns))
    
    cat_features = [
        "item_id",
        "dept_id",
        "cat_id",
        "store_id",
        "state_id",
        "event_name_1",
        "event_type_1",
        "event_name_2",
        "event_type_2",
        # time features.
        "year",
        "month",
        "week",
        "day",
        "dayofweek",
    ]

    available_cat_features = [f for f in cat_features if f in data.columns]
    
    ########################
    #### GET GENERATORS ####
    ########################
    # going to evaluate with the last 28 days
    test = data[(data['date'] > '2016-04-24')]
    
    # get generators
    labels = ['demand']
    batch_size = 1024
    test_batch_creator = BatchCreator(test, features, labels, shuffle=False, ensure_all_samples=True,
                                      categorical_features=available_cat_features, batch_size=batch_size, 
                                      check_nan=False)
    
    # determine model input shape
    x, y = next(test_batch_creator.flow())
    INP_SHAPE = x[0].shape
    
    # make losses
    losses = get_pinball_losses()
    
    print("Input size level {}: {}".format(level, INP_SHAPE))
    
    
    #########################
    #### GET PREDICTIONS ####
    #########################
    print("Reading model..")
    model = models_base[level]
    model.load_weights(models_weights[level])
    df = evaluate_model(model, ref, test_batch_creator, calendar, quantiles, data_dir, level, 
                                 validation_set=True, verbose=False)
    
    collected_dfs[level] = df
    
    del data, test, test_batch_creator
    gc.collect()

Starting level 1
Reading from file..
Input size level 1: (178,)
Reading model..
Predicting...
Denormalising...
Warping predictions...
Starting level 2
Reading from file..
Input size level 2: (181,)
Reading model..
Predicting...
Denormalising...
Warping predictions...
Starting level 3
Reading from file..
Input size level 3: (191,)
Reading model..
Predicting...
Denormalising...
Warping predictions...
Starting level 4
Reading from file..
Input size level 4: (181,)
Reading model..
Predicting...
Denormalising...
Warping predictions...
Starting level 5
Reading from file..
Input size level 5: (188,)
Reading model..
Predicting...
Denormalising...
Warping predictions...
Starting level 6
Reading from file..
Input size level 6: (184,)
Reading model..
Predicting...
Denormalising...
Warping predictions...
Starting level 7
Reading from file..
Input size level 7: (191,)
Reading model..
Predicting...
Denormalising...
Warping predictions...
Starting level 8
Reading from file..
Input size level 8: (194,

### Combine predictions and create submission

In [9]:
preds = pd.concat(collected_dfs.values())
preds.head()

,d_1914,d_1915,d_1916,d_1917,d_1918,d_1919,d_1920,d_1921,d_1922,d_1923,...,d_1935,d_1936,d_1937,d_1938,d_1939,d_1940,d_1941,id,quantile,level
0,33139.988599,31227.096495,31442.217990,31207.094156,35241.970168,42979.785673,43391.854329,36996.262308,37389.413150,33466.038670,...,35783.274741,32928.366235,32667.217191,32407.902974,36121.090894,44504.369786,44649.648500,Total_X_0.005_evaluation,0.005,1
1,33990.522861,31924.402264,32157.619652,32071.805693,36127.461630,44123.445437,44535.319697,37863.217927,38198.933237,34143.840026,...,36758.055419,33725.728037,33396.482367,33127.206201,37012.792800,45647.333817,45779.021853,Total_X_0.025_evaluation,0.025,1
2,35352.001793,33040.604426,33302.783429,33455.977817,37544.906188,45954.142080,46365.709399,39250.981264,39494.760842,35228.819440,...,38318.425474,35002.095565,34563.844819,34278.621348,38440.168613,47476.925471,47586.851371,Total_X_0.165_evaluation,0.165,1
3,35766.464591,33380.402055,33651.400503,33877.353614,37976.407637,46511.448524,46922.920350,39673.448408,39889.241763,35559.112121,...,38793.437553,35390.652520,34919.218012,34629.141459,38874.695139,48033.894280,48137.197070,Total_X_0.250_evaluation,0.250,1
4,36697.277637,34143.526263,34434.322423,34823.680836,38945.479206,47763.056123,48174.316501,40622.231160,40775.169764,36300.886859,...,39860.225582,36263.276322,35717.316473,35416.336684,39850.560342,49284.741347,49373.167717,Total_X_0.500_evaluation,0.500,1


In [10]:
preds.shape

(385560, 31)

In [11]:
preds.to_csv(sub_dir + "fold0_levels_1_12.csv")

In [14]:
fn = create_uncertainty_submission_from_ref_format(
            preds, submission_dir=sub_dir, filename="fold0_try1", add_timestamp=True)

In [15]:
!kaggle competitions submit -c m5-forecasting-uncertainty -f $fn -m "Fold 0 try 1 predictions"

100%|███████████████████████████████████████| 38.1M/38.1M [00:50<00:00, 796kB/s]
Successfully submitted to M5 Forecasting - Uncertainty

In [13]:
from flow import create_uncertainty_submission_from_ref_format